# 5分钟学会Python爬取整个网站

爬取网站的步骤：
1. 设定爬取目标
   * 目标网站：我自己的博客，疯狂的蚂蚁 http://www.crazyant.net
   * 目标数据：所有博客文章的 - 链接、标题、标签
2. 分析目标网站
   * 待爬取页面：http://www.crazyant.net/page/1  ~ http://www.crazyant.net/page/24
   * 待爬取数据：HTML元素中的h2 class=entry-title下的超链接的标题和链接，标签列表
3. 批量下载HTML
   * 使用requests库实现下载，官网：https://2.python-requests.org//zh_CN/latest/user/quickstart.html
4. 实现HTML解析，得到目标数据
   * 使用BeautifulSoup库解析，官网：https://beautifulsoup.readthedocs.io/zh_CN/v4.4.0/
5. 将结果数据存储
   * 可以使用json.dumps把这个数据序列化存储

In [1]:
import requests
from bs4 import BeautifulSoup
import pprint
import json

## 1、下载所有的页面的HTML

In [2]:
def download_all_htmls():
    """
    下载所有列表页面的HTML，用于后续的分析
    """
    htmls = []
    for idx in range(24):
        url = f"http://www.crazyant.net/page/{idx+1}"
        print("craw html:", url)
        r = requests.get(url)
        if r.status_code != 200:
            raise Exception("error")
        htmls.append(r.text)
    return htmls

In [3]:
# 执行爬取
htmls = download_all_htmls()

craw html: http://www.crazyant.net/page/1
craw html: http://www.crazyant.net/page/2
craw html: http://www.crazyant.net/page/3
craw html: http://www.crazyant.net/page/4
craw html: http://www.crazyant.net/page/5
craw html: http://www.crazyant.net/page/6
craw html: http://www.crazyant.net/page/7
craw html: http://www.crazyant.net/page/8
craw html: http://www.crazyant.net/page/9
craw html: http://www.crazyant.net/page/10
craw html: http://www.crazyant.net/page/11
craw html: http://www.crazyant.net/page/12
craw html: http://www.crazyant.net/page/13
craw html: http://www.crazyant.net/page/14
craw html: http://www.crazyant.net/page/15
craw html: http://www.crazyant.net/page/16
craw html: http://www.crazyant.net/page/17
craw html: http://www.crazyant.net/page/18
craw html: http://www.crazyant.net/page/19
craw html: http://www.crazyant.net/page/20
craw html: http://www.crazyant.net/page/21
craw html: http://www.crazyant.net/page/22
craw html: http://www.crazyant.net/page/23
craw html: http://ww

In [4]:
htmls[0]

'<!DOCTYPE html><html lang="zh-CN" class="no-js"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width"><link rel="profile" href="http://gmpg.org/xfn/11"><link rel="pingback" href="http://www.crazyant.net/xmlrpc.php"> <!--[if lt IE 9]> <script src="http://www.crazyant.net/wp-content/themes/twentyfifteen/js/html5.js"></script> <![endif]--> <script>(function(html){html.className = html.className.replace(/\\bno-js\\b/,\'js\')})(document.documentElement);</script> <title>疯狂的蚂蚁 &#8211; 视频公众号：蚂蚁学Python</title><link rel=\'dns-prefetch\' href=\'//cdn.bibblio.org\' /><link rel="alternate" type="application/rss+xml" title="疯狂的蚂蚁 &raquo; Feed" href="http://www.crazyant.net/feed" /><link rel="alternate" type="application/rss+xml" title="疯狂的蚂蚁 &raquo; 评论Feed" href="http://www.crazyant.net/comments/feed" /> <!-- managing ads with Advanced Ads – https://wpadvancedads.com/ --><script>advanced_ads_ready=function(){var fns=[],listener,doc=typeof document==="object"&&document,hack

## 2、解析HTML得到数据

In [6]:
def parse_single_html(html):
    """
    解析单个HTML，得到数据
    @return list({"link", "title", [label]})
    """
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all("article")
    datas = []
    for article in articles:
        # 查找超链接
        title_node = (
            article
            .find("h2", class_="entry-title")
            .find("a")
        )
        title = title_node.get_text()
        link = title_node["href"]
        
        # 查找标签列表
        tag_nodes = (
            article
            .find("footer", class_="entry-footer")
            .find("span", class_="tags-links")
            .find_all("a")
        )
        tags = [tag_node.get_text() for tag_node in tag_nodes]
        datas.append(
            {"title":title, "link":link, "tags":tags}
        )
    return datas



In [7]:
pprint.pprint(parse_single_html(htmls[0]))

[{'link': 'http://www.crazyant.net/2469.html',
  'tags': ['java', 'spark'],
  'title': 'Spark使用Java开发遇到的那些类型错误'},
 {'link': 'http://www.crazyant.net/2454.html',
  'tags': ['pandas', 'python', 'sklearn', '推荐系统'],
  'title': '推荐系统：实现文章相似推荐的简单实例'},
 {'link': 'http://www.crazyant.net/2447.html',
  'tags': ['item2vec', 'java', '推荐系统'],
  'title': 'Spark使用word2vec训练item2vec实现内容相关推荐'},
 {'link': 'http://www.crazyant.net/2434.html',
  'tags': ['pandas', 'python'],
  'title': 'Pandas中对轴axis=0和axis=1的理解'},
 {'link': 'http://www.crazyant.net/2419.html',
  'tags': ['echarts', 'flask', 'pyecharts', 'python'],
  'title': 'Flask使用Pyecharts在单个页面展示多个图表'},
 {'link': 'http://www.crazyant.net/2404.html',
  'tags': ['程序人生'],
  'title': '听樊登的《非暴力沟通》'},
 {'link': 'http://www.crazyant.net/2367.html',
  'tags': ['java', 'python', 'tensorflow'],
  'title': 'Java和Python使用Grpc访问Tensorflow的Serving代码'},
 {'link': 'http://www.crazyant.net/2351.html',
  'tags': ['推荐系统'],
  'title': '推荐系统：怎样实现内容相似推荐'},
 {'link': 'http

In [8]:
# 执行所有的HTML页面的解析
all_datas = []
for html in htmls:
    all_datas.extend(parse_single_html(html))

In [11]:
all_datas

[{'title': 'Spark使用Java开发遇到的那些类型错误',
  'link': 'http://www.crazyant.net/2469.html',
  'tags': ['java', 'spark']},
 {'title': '推荐系统：实现文章相似推荐的简单实例',
  'link': 'http://www.crazyant.net/2454.html',
  'tags': ['pandas', 'python', 'sklearn', '推荐系统']},
 {'title': 'Spark使用word2vec训练item2vec实现内容相关推荐',
  'link': 'http://www.crazyant.net/2447.html',
  'tags': ['item2vec', 'java', '推荐系统']},
 {'title': 'Pandas中对轴axis=0和axis=1的理解',
  'link': 'http://www.crazyant.net/2434.html',
  'tags': ['pandas', 'python']},
 {'title': 'Flask使用Pyecharts在单个页面展示多个图表',
  'link': 'http://www.crazyant.net/2419.html',
  'tags': ['echarts', 'flask', 'pyecharts', 'python']},
 {'title': '听樊登的《非暴力沟通》',
  'link': 'http://www.crazyant.net/2404.html',
  'tags': ['程序人生']},
 {'title': 'Java和Python使用Grpc访问Tensorflow的Serving代码',
  'link': 'http://www.crazyant.net/2367.html',
  'tags': ['java', 'python', 'tensorflow']},
 {'title': '推荐系统：怎样实现内容相似推荐',
  'link': 'http://www.crazyant.net/2351.html',
  'tags': ['推荐系统']},
 {'title': 'Fla

In [12]:
len(all_datas)

239

## 3、将结果输出存储
1. MySQL
2. 本地JSON文件

In [13]:
with open("all_article_links.json", "w") as fout:
    for data in all_datas:
        fout.write(json.dumps(data, ensure_ascii=False)+"\n")